In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Transaction, Participation
from player.models import Player
import random

In [2]:

random_index = {
    'money': [10000,10000,15000,20000,25000,30000,35000,40000,45000,50000],
    'health_score': [20, 40, 40, 60, 60, 80, 80, 100, 100, 120],
    'skill_score': [20, 20, 40, 40, 60, 60, 80, 100, 120, 130],
    'growth_score': [5, 5, 5, 10, 10, 15, 20, 25, 30, 35],
    'relationship_score': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'academic_level': [0, 0, 0, 0, 1, 1, 1, 2, 2, 4]
}
def create_player(u):
    Player.objects.create(
        user=u,
        born_money=random.choice(random_index['money']),
        born_health_score=random.choice(random_index['health_score']),
        born_skill_score=random.choice(random_index['skill_score']),
        born_growth_score=random.choice(random_index['growth_score']),
        born_relationship_score=random.choice(random_index['relationship_score']),
        born_academic_level=random.choice(random_index['academic_level']),
        born_steps=15
    )

In [3]:
# delete all existing players
# Player.objects.all().delete()

In [3]:
students = User.objects.filter(id__gte=290, id__lte=294)
# others = User.objects.filter(id__gt=500)
for u in students:
    create_player(u)
# for u in others:
#     create_player(u)

In [2]:
Player.objects.all().update(born_defect='')

730

In [3]:
# student google sheet
import pandas as pd
student_url = 'https://docs.google.com/spreadsheets/d/' + \
                   '1whM8CflvlkdVNdfdaaAAuzVn4YAMmCrws48Djrt8ONw' + \
                   '/export?gid=1768285988&format=csv'
student_df = pd.read_csv(student_url)
student_df = student_df[student_df['special'].notna()]
print(student_df.head())

   id  組別 級別       school  學校人數 學生姓名 last_name first_name   班級 男/女  room  \
1   2   1  A   禮賢會彭學高紀念中學  31.0  馬苡晴         馬         苡晴   4C   女  C201   
2   3   1  A  香港布廠商會朱石麟中學   2.0  陳楚澄         陳         楚澄   F3   女  B204   
4   5   2  A   禮賢會彭學高紀念中學   2.0  劉鍵澄         劉         鍵澄   5A   女  C201   
6   7   2  A  香港布廠商會朱石麟中學   3.0  陳嘉穎         陳         嘉穎   F3   男  D205   
8  10   3  A         優才書院   2.0  許恩慈         許         恩慈  10A   女  B201   

  special  
1  先天性心臟病  
2      癌症  
4      癌症  
6      失明  
8      失明  


In [4]:
for i, r in student_df.iterrows():
    u = User.objects.get(id=r['id'])
    p = Player.objects.get(user=u)
    p.born_defect = r['special']
    p.save()